In [ ]:
#@title Setup & Clone Repo
!git clone https://github.com/facebookresearch/LivePortrait.git
%cd LivePortrait

!pip install -r requirements.txt
!pip install torch torchvision --quiet

print("Setup done ✅")

In [ ]:
#@title Import Libraries and Load Model
import torch
import time
from PIL import Image
import matplotlib.pyplot as plt

# Replace below with actual LivePortrait model import and load
class LivePortraitModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Init model layers here

    def forward(self, x):
        # Forward pass here
        return x  # dummy pass through

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LivePortraitModel()
# model.load_state_dict(torch.load('path_to_weights.pth'))
model.to(device)
model.eval()

print(f"Using device: {device}")
print("Model loaded ✅")

In [ ]:
#@title Preprocessing and Postprocessing
def preprocess(image_path):
    image = Image.open(image_path).convert('RGB')
    image = image.resize((256,256))  # adjust size if needed
    image = torch.tensor([[[pixel/255 for pixel in channel] for channel in image.split()]]).float()
    return image.to(device)

def postprocess(output_tensor):
    output_tensor = output_tensor.squeeze().cpu().clamp(0,1)
    output_np = output_tensor.permute(1,2,0).numpy()
    return Image.fromarray((output_np * 255).astype('uint8'))

print("Preprocessing ready ✅")

In [ ]:
#@title Original Inference & Timing
input_image_path = 'path_to_sample_input.jpg'  # replace with your image path
input_tensor = preprocess(input_image_path)

start_time = time.time()
with torch.no_grad():
    output = model(input_tensor)
end_time = time.time()

original_time = end_time - start_time
print(f"Original inference time: {original_time:.4f} seconds")

plt.imshow(postprocess(output))
plt.title("Original Output")
plt.axis('off')
plt.show()

In [ ]:
#@title Optimized Inference with Mixed Precision
from torch.cuda.amp import autocast

start_time = time.time()
with torch.no_grad():
    with autocast():
        output_opt = model(input_tensor)
end_time = time.time()

optimized_time = end_time - start_time
print(f"Optimized inference time (mixed precision): {optimized_time:.4f} seconds")

plt.imshow(postprocess(output_opt))
plt.title("Optimized Output (Mixed Precision)")
plt.axis('off')
plt.show()

# Summary of Optimizations

- Used mixed precision inference (`torch.cuda.amp`) to speed up inference and reduce memory.
- Ensured model eval mode and no-grad context.
- Kept data on GPU to avoid transfer overhead.

# Performance Comparison

| Metric               | Original        | Optimized (Mixed Precision) |
|----------------------|-----------------|-----------------------------|
| Inference Time (sec)  | original_time   | optimized_time              |
| GPU Memory Usage      | Baseline        | Reduced                    |
| Output Quality       | Baseline        | Comparable (visually similar)|

# Future Optimizations

- Convert to TorchScript or ONNX for faster inference.
- Model pruning or quantization.
- Layer fusion or CUDA kernel tuning.
- Batch size tuning for throughput.

---

Replace placeholders with your actual model, weights, and image paths. Run all cells on GPU in Colab and fill in timings/output quality.

If you want me to generate a downloadable `.ipynb` notebook file or help with exact repo code integration, just say the word!